In [437]:
import random

import numpy as np
import pandas as pd
from scipy.stats import f_oneway
from scipy.spatial.distance import cdist
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import heapq as hq

In [438]:
# 计算方差窗口的大小
def computeWinSize(length, b):
    if 0 < length <= b:
        return 1
    else:
        return length // b

In [439]:
# 计算标准差数组
def computeStdArr(series, b):
    l = len(series)
    # 窗口下取整
    w = computeWinSize(l, b)
    # print(w)
    res = []
    for i in range(w, l - w):
        win_arr = [0 for i in range(2 * w + 1)]
        cnt = 0
        for j in range(i - w, w + i + 1):
            win_arr[cnt] = series[j]
            cnt += 1
        std = np.std(win_arr, ddof=0)
        res.append(std)
    return res

In [440]:
# 选择L*α最大的点的下标,，保留下标
def selectLargePoint(series, a):
    l = len(series)
    # 生成l*a个
    res = hq.nlargest(int(l * a), range(l), key=lambda x: series[x])
    return res

In [441]:
# 判断是否为波峰波谷
def isPeakOrValley(std_arr, index, b):
    win_size = computeWinSize(len(std_arr), b)
    flag = False
    if win_size == 1:
        if ((std_arr[index] > std_arr[index - 1] and std_arr[index] > std_arr[index + 1]) or (
                std_arr[index] < std_arr[index - 1] and std_arr[index] < std_arr[index + 1])):
            flag = True
    elif win_size == 2:
        if ((std_arr[index] >= std_arr[index - 1] and std_arr[index] > std_arr[index - 2] and std_arr[index] >= std_arr[
            index + 1] and std_arr[index] > std_arr[index + 2])
                or
                (std_arr[index] <= std_arr[index - 1] and std_arr[index] < std_arr[index - 2] and std_arr[index] <=
                 std_arr[
                     index + 1]) and std_arr[index] < std_arr[index + 2]):
            flag = True
    elif win_size == 3:
        if ((std_arr[index] >= std_arr[index - 1] and std_arr[index] > std_arr[index - 2] and std_arr[index] > std_arr[
            index - 3]
             and std_arr[index] >= std_arr[index + 1] and std_arr[index] > std_arr[index + 2] and std_arr[index] >
             std_arr[
                 index + 3])
                or
                (std_arr[index] <= std_arr[index - 1] and std_arr[index] < std_arr[index - 2] and std_arr[index] <
                 std_arr[index - 3]) and std_arr[index] <= std_arr[index + 1] and std_arr[index] < std_arr[
                    index + 2] and std_arr[index] < std_arr[index + 3]):
            flag = True
    elif win_size == 4:
        if ((std_arr[index] >= std_arr[index - 1] and std_arr[index] > std_arr[index - 2] and std_arr[index] > std_arr[
            index - 3] and std_arr[index] > std_arr[index - 4]
             and std_arr[index] >= std_arr[index + 1] and std_arr[index] > std_arr[index + 2] and std_arr[index] >
             std_arr[
                 index + 3] and std_arr[index] > std_arr[index + 4])
                or
                (std_arr[index] <= std_arr[index - 1] and std_arr[index] < std_arr[index - 2] and std_arr[index] <
                 std_arr[
                     index - 3] and std_arr[index] < std_arr[index - 4]
                 and std_arr[index] <= std_arr[index + 1] and std_arr[index] < std_arr[index + 2] and std_arr[index] <
                 std_arr[index + 3] and std_arr[index] < std_arr[index + 4])):
            flag = True
    else:
        if ((std_arr[index] > std_arr[index - 1] and std_arr[index] > std_arr[index - 2] and std_arr[index] > std_arr[
            index - 3] and std_arr[index] > std_arr[index - 4] and std_arr[index] > std_arr[index - 5]
             and std_arr[index] > std_arr[index + 1] and std_arr[index] > std_arr[index + 2] and std_arr[index] >
             std_arr[
                 index + 3] and std_arr[index] > std_arr[index + 4] and std_arr[index] > std_arr[index + 5])
                or
                (std_arr[index] < std_arr[index - 1] and std_arr[index] < std_arr[index - 2] and std_arr[index] <
                 std_arr[
                     index - 3] and std_arr[index] < std_arr[index - 4] and std_arr[index] < std_arr[index - 5]
                 and std_arr[index] < std_arr[index + 1] and std_arr[index] < std_arr[index + 2] and std_arr[index] <
                 std_arr[
                     index + 3] and std_arr[index] < std_arr[index + 4] and std_arr[index] < std_arr[index + 5])):
            flag = True
    return flag

In [442]:
# 找到所有合适的区间
def fitPoint(stdA, indexA, b):
    w = computeWinSize(len(stdA), b)
    l = len(indexA)
    res = []
    for i in range(0, l):
        val = indexA[i]
        if w <= val < l - w:
            if isPeakOrValley(stdA, val, b):
                res.append(val + 2 * w + 1)
    return res

In [443]:
# 提取特征关键点
def extractFeaturePoint(arr, a, b):
    stdArrOne = computeStdArr(arr, b)
    largePoint = selectLargePoint(stdArrOne, a)
    indexArr = fitPoint(stdArrOne, largePoint, b)
    return indexArr

In [444]:
# 判断有几个关键点与它相近
def closeNums(keyList,key,threshold):
    cnt = 0
    cnt += keyList.count(key)
    for i in range(1,threshold):
        a = key + i
        b = key - i
        cnt += keyList.count(a)
        cnt += keyList.count(b)
    return cnt

In [445]:
class keyPoint:
    def __init__(self,key,cnt,dim):
        self.key = key
        self.cnt = cnt
        self.dim = dim

In [446]:
class smallKeyPoint:
    def __init__(self,key,dim):
        self.key = key
        self.dim = dim

In [447]:
# 找到较为相近的关键点
def selectKeys(data,alp,blt,howclose,orgin_dim_nums):
    # 原始data顺序 样本数 维度数 序列长度
    dim_nums = data.shape[1]
    sample_nums = data.shape[0]
    all_keys = []
    for sm in tqdm(range(sample_nums)):
        one_sample  = data[sm]
        ################ 原始序列 #####################

        # 每个维度的关键点下标集合
        key_points = []
        # 所为维度下的关键点下标
        ex_key_points = []
        for dn in range(orgin_dim_nums):
            # keys 关键点的下标
            keys = extractFeaturePoint(one_sample[dn],alp,blt)
            key_points.append(keys)
        # 有一部分在构建的序列上

        for dn in range(orgin_dim_nums):
            # 一个维度上的关键下标
            kl = key_points[dn]
            # 去掉自己维度的下标
            tmp_key_points = key_points[:]
            tmp_key_points.remove(kl)
            key_list = []
            for i in tmp_key_points:
                key_list.extend(i)

            ekl = []
            for k in kl:
                cnt = closeNums(key_list,k,howclose)
                ekl.append(keyPoint(k,cnt,dn))
            ex_key_points.extend(ekl)

        ex_key_points.sort(key=lambda x:x.cnt,reverse=True)
        # 筛选
        res_keys = ex_key_points[:]
        res = 0
        for k in res_keys:
            if k.cnt >= res_keys[0].cnt * 0.9:
                res += 1
        res_keys = res_keys[:res]

        ################ 转换序列 ####################

        # 每个维度的关键点下标集合
        key_points = []
        # 所为维度下的关键点下标
        ex_key_points = []
        for dn in range(orgin_dim_nums,dim_nums):
            # keys 关键点的下标
            keys = extractFeaturePoint(one_sample[dn],alp,blt)
            key_points.append(keys)
        # 有一部分在构建的序列上

        for dn in range(orgin_dim_nums,dim_nums):
            # 一个维度上的关键下标
            kl = key_points[dn-orgin_dim_nums]
            # 去掉自己维度的下标
            tmp_key_points = key_points[:]
            tmp_key_points.remove(kl)
            key_list = []
            for i in tmp_key_points:
                key_list.extend(i)

            ekl = []
            for k in kl:
                cnt = closeNums(key_list,k,howclose)
                ekl.append(keyPoint(k,cnt,dn))
            ex_key_points.extend(ekl)

        ex_key_points.sort(key=lambda x:x.cnt,reverse=True)
        # 筛选
        res_keys2 = ex_key_points[:]
        res = 0
        for k in res_keys2:
            if k.cnt >= res_keys2[0].cnt * 0.9:
                res += 1
        res_keys2 = res_keys2[:res]



        all_keys.append(res_keys + res_keys2)
        # all_keys.append(res_keys)
    return all_keys

In [448]:
class shapelet:
    def __init__(self,keyPoint,ylt,st,ed,length,val,sample):
        # keyPoint 由那个keyPoint生成
        # ylt 松弛是多少
        # st 起始位置
        # ed 终止位置
        # val shapelet在序列中的实际的值
        self.dist = None
        self.quality = None
        self.keyPoint = keyPoint
        self.ylt = ylt
        self.st = st
        self.ed = ed
        self.length = length
        self.val = val
        self.sample = sample
    def setQuality(self,quality):
        self.quality = quality
    def setDis(self,dist):
        self.dist = dist

In [449]:
def generateCandidates(s,sm,l,keyP,ylt):
    # sm 为单个样本[维度，长度]
    # keyP 应该为一个keyPoint
    # ylt为松弛参数
    candidate = []
    if l % 2 == 1:
        l = l // 2
        dim = keyP.dim
        s = s[dim]
        for i in range(-ylt,ylt+1):
            # 从中心点两侧的松弛长度进行计算
            if keyP.key - i - l < 0:
                continue
            left = keyP.key - i - l
            if keyP.key - i + l + 1 >= len(s):
                continue
            right = keyP.key - i + l + 1
            spt = shapelet(keyPoint=keyP,ylt=ylt,st=left,ed=right - 1,length=right-left,val=s[left:right],sample=sm)
            candidate.append(spt)
    else:
        l = l // 2
        dim = keyP.dim
        s = s[dim]
        for i in range(-ylt,ylt+1):
            # 从中心点两侧的松弛长度进行计算
            if keyP.key - i - l < 0:
                continue
            left = keyP.key - i - l
            if keyP.key - i + l >= len(s):
                continue
            right = keyP.key - i + l
            spt = shapelet(keyPoint=keyP,ylt=ylt,st=left,ed=right - 1,length=right-left,val=s[left:right],sample=sm)
            candidate.append(spt)
            if keyP.key - i - l + 1< 0:
                continue
            left = keyP.key - i - l + 1
            if keyP.key - i + l + 1 >= len(s):
                continue
            right = keyP.key - i + l + 1
            spt = shapelet(keyPoint=keyP,ylt=ylt,st=left,ed=right - 1,length=right-left,val=s[left:right],sample=sm)
            candidate.append(spt)
    return candidate

In [450]:
import collections

In [451]:
def findShapeletCandidates(data, lmin, lmax, keys, ylt, odim):
    sample_nums = data.shape[0]
    candidate = []
    for sm in tqdm(range(sample_nums)):
        s = data[sm]
        keyList = keys[sm]
        for k in keyList:
            # lmin - lmax
            for l in range(lmin, lmax + 1):
                candidate.extend(generateCandidates(s, sm, l, k, ylt))
    # print(len(candidate))
    dt = collections.defaultdict(list)
    for cd in candidate:
        dt[cd.length].append(cd)

    # 对dt进行剪枝
    v = dt.keys()
    # print(len(dt[3]))

    # 对于每个长度
    for k in v:
        ls = dt[k]
        tmpls = []
        #     每个维度都选几个
        res = collections.defaultdict(list)
        for i in ls:
            # print(type(i))
            res[i.keyPoint.dim].append(i)
        dims = list(res.keys())
        org_dim_list = []
        convert_dim_list = []
        for i in dims:
            if i >= odim:
                convert_dim_list.append([i, len(res[i])])
            else:
                org_dim_list.append([i, len(res[i])])

        org_dim_list.sort(key=lambda x: x[1], reverse=True)
        convert_dim_list.sort(key=lambda x: x[1], reverse=True)


        org_pre_dim = int(len(org_dim_list) * 0.3) if int(len(org_dim_list) * 0.3) > 0 else 1
        org_pre_dims = org_dim_list[:org_pre_dim + 1]
        org_aft_dims = org_dim_list[org_pre_dim + 1:]
        if len(org_pre_dims) > 1:
            org_pre_dims = random.sample(org_pre_dims, min(org_pre_dim,int(org_pre_dim * 0.7) + 1 if int(org_pre_dim * 0.7) > 0 else 1))
        if len(org_aft_dims) > 1:
            org_aft_dims = random.sample(org_aft_dims, min(len(org_aft_dims),int(len(org_aft_dims) * 0.3) + 1 if int(len(org_aft_dims) * 0.3) > 0 else 1))
        print(org_pre_dims,org_aft_dims)
        #
        convert_pre_dim = int(len(convert_dim_list) * 0.3) if int(len(convert_dim_list) * 0.3) > 0 else 1
        convert_pre_dims = convert_dim_list[:convert_pre_dim + 1]
        convert_aft_dims = convert_dim_list[convert_pre_dim + 1:]
        if len(convert_pre_dims) > 1:
            convert_pre_dims = random.sample(convert_pre_dims, min(convert_pre_dim,int(convert_pre_dim * 0.7)+1 if int(convert_pre_dim * 0.7) > 0 else 1 ))
        if len(convert_aft_dims) > 1:
            convert_aft_dims = random.sample(convert_aft_dims, min(len(convert_aft_dims),int(len(convert_aft_dims) * 0.3) if int(len(convert_aft_dims) * 0.3) > 0 else 1  ))

        print(convert_pre_dims,convert_aft_dims)

        dims_list =  org_pre_dims + org_aft_dims + convert_pre_dims + convert_aft_dims
        dims = []


        for i in dims_list:
            dims.append(i[0])
        dims = list(set(dims))
        # print(dims)

        # # 筛选维度
        for i in dims:

            tmp = res[i]
            # 尽量选取每个样本
            newdt = collections.defaultdict(list)
            for x in tmp:
                newdt[x.sample].append(x)
            klist = list(newdt.keys())

            indx = 0
            smp = []
            while len(smp) <= min(50, len(tmp)):
                index = klist[indx]
                indx = (indx + 1) % len(klist)
                rc = random.choice(newdt[index])
                # 防止死循环
                cntime = 0
                while rc in smp:
                    cntime += 1
                    rc = random.choice(newdt[index])
                    if cntime == 50:
                        break
                smp.append(rc)
            # smp = random.sample(tmp,min(50,len(tmp)))
            tmpls.extend(smp)

        dt[k] = tmpls
    return dt

In [452]:
def findDistances(c,l,data):
    # 查找获选shapelet在每个样本上的最小值
    ds = []
    dim = c.keyPoint.dim
    data = np.transpose(data,axes=(1,0,2))
    sers = data[dim]
    for s in sers:
        cp = [s[i : i + l] for i in range(0, len(s)-l+1, 1)]
        ds.append(cdist([c.val],cp,metric='seuclidean').min())
    return np.array(ds)

In [453]:
def assessCandidate(ds,label):
    class_groups = []
    for c in np.unique(label):
        class_groups.append(ds[label==c].tolist())
#  返回f值
    return f_oneway(*class_groups).statistic

In [454]:
def sortByQuality(shapelets):
    return sorted(shapelets, key=lambda s: s.quality,reverse=True)

In [455]:
def sortByDist(shapelets):
    return sorted(shapelets, key=lambda s: s.dist,reverse=True)


In [456]:
class interval:
    def __init__(self,st,ed):
        self.st = st
        self.ed = ed
    def isInclude(self,a):
        # 是否被a包含
        if self.st >= a.st and self.ed <= a.ed:
            return True

In [457]:
def removeSelfSimilar(shapelets):
    ts = shapelets[:][::-1]
    it = []
    for x in ts:
        it.append(interval(x.st,x.ed))
    removeIdx = []
    l = len(it)
    # 修改将重叠改为包含
    for i in range(l):
        for j in range(i+1,l):
            if it[i].isInclude(it[j]):
                removeIdx.append(i)
                break
    res = []
    for i in range(l):
        if i not in removeIdx:
            res.append(ts[i])


    return res[::-1]

In [458]:
def merge(k,kShapelets,shapelets):
    total_shapelets = kShapelets + shapelets
    return sortByQuality(total_shapelets)[:k]

In [459]:
def calDis(ds):
    return sum(ds)

In [460]:


def findShapelets(data,label,dt,k):
    kShapelets1 = []
    kShapelets2 = []
    # l 为长度
    for l in dt.keys():
        # cd 是单个长度的所有样本和维度下的shaplet候选
        cd = dt[l]
        if len(cd) == 0:
            continue
        for i,c in enumerate(tqdm(cd)):
            ds = findDistances(c,l,data)
            quality = assessCandidate(ds,label)
            dist = calDis(ds)
            cd[i].setQuality(quality)
            cd[i].setDis(dist)
        cd1 = sortByQuality(cd)
        # print(len(cd))
        cd1 = removeSelfSimilar(cd1)
        # print(len(cd))
        kShapelets1 = merge(k,kShapelets1,cd1)
        cd2 = sortByDist(cd)
        cd2 = removeSelfSimilar(cd2)
        # print(len(cd))
        kShapelets2 = merge(k,kShapelets2,cd2)
    kShapelets = list(set(kShapelets1 + kShapelets2))
    return kShapelets

In [461]:
def load_raw_ts(path, dataset):
    path = path + "raw//" + dataset + "//"
    # 训练集
    x_train = np.load(path + 'X_train.npy')
    x_train = np.transpose(x_train, axes=(0, 2, 1))
    x_test = np.load(path + 'X_test.npy')
    x_test = np.transpose(x_test, axes=(0, 2, 1))
    y_train = np.load(path + 'y_train.npy')
    y_test = np.load(path + 'y_test.npy')
    labels = np.concatenate((y_train, y_test), axis=0)
    nclass = int(np.amax(labels)) + 1
    return x_train, x_test, y_train.reshape(-1), y_test.reshape(-1), nclass

In [462]:
# 对于每个样本生成差异序列序列
def genDiffSeries(data):
    sample_nums = data.shape[0]
    dim_nums = data.shape[1]
    series_length = data.shape[2]
    gen_data = []
    for sn in range(sample_nums):
        sm = data[sn]
        res = []
        # 先将原始的每个维度的加进入
        for i in range(dim_nums):
            res.append(sm[i])
        # 再计算每个减法维度，然后依次加进去
        for i in range(dim_nums-1):
            for j in range(i+1,dim_nums):
                s1 = sm[i]
                s2 = sm[j]
                news = [0 for _ in range(series_length)]
                for k in range(series_length):
                    news[k] = s1[k] - s2[k]
                res.append(np.array(news))
        gen_data.append(np.array(res))
    return np.array(gen_data)


In [463]:
# target_train 为标签 data_train 为序列
data_train, data_test, target_train, target_test, nclass = load_raw_ts("D://tmppro//data//", 'StandWalkJump')
sample_nums = data_train.shape[0]
dim_nums = data_train.shape[1]
series_length = data_train.shape[2]
test_nums = data_test.shape[0]
lmin = 3
lmax = 13
# 选取shapelet的数量
k = 150
alp = 0.3
# k = 20
# alp = random.choice([0.3, 0.4])
# blt = random.choice([100, 90, 80, 70, 60, 50])
blt = 50
# ylt = random.choice([5,6,7,8])
ylt = 4
dis = 2

print(data_train.shape)
a = genDiffSeries(data_train)
print(a.shape)


(12, 4, 2500)
(12, 10, 2500)


In [464]:
keys = selectKeys(a, alp, blt, 2,dim_nums)
print('select keys')

100%|██████████| 12/12 [00:21<00:00,  1.77s/it]

select keys


In [465]:
# print(len(keys[200]))

In [466]:
dt = findShapeletCandidates(a, lmin, lmax, keys, ylt,dim_nums)
print('select candi')


100%|██████████| 12/12 [00:00<00:00, 256.03it/s]

[[3, 135]] [[0, 117]]
[[4, 63]] [[6, 36]]
[[3, 270]] [[0, 234]]
[[5, 162]] [[8, 90]]
[[1, 135]] [[0, 117]]
[[5, 81]] [[6, 36]]
[[3, 270]] [[0, 234]]
[[5, 162]] [[6, 72]]
[[3, 135]] [[0, 117]]
[[4, 63]] [[9, 27]]
[[1, 270]] [[2, 234]]
[[4, 126]] [[6, 72]]
[[1, 135]] [[2, 117]]
[[4, 63]] [[8, 45]]
[[1, 270]] [[0, 234]]
[[5, 162]] [[6, 72]]
[[3, 135]] [[0, 117]]
[[5, 81]] [[9, 27]]
[[1, 270]] [[2, 234]]
[[4, 126]] [[9, 54]]
[[3, 135]] [[2, 117]]
[[5, 81]] [[8, 45]]
select candi


In [467]:
kshapelets = findShapelets(a, target_train, dt, k)
print('find shapelet')


  7%|▋         | 14/190 [00:00<00:05, 31.77it/s]c:\users\ydssx\appdata\local\programs\python\python38\lib\site-packages\scipy\stats\stats.py:3650: F_onewayConstantInputWarning: Each of the input arrays is constant;the F statistic is not defined or infinite
  warnings.warn(F_onewayConstantInputWarning())
100%|██████████| 199/199 [00:06<00:00, 29.87it/s]

find shapelet


In [468]:
for i in kshapelets:
    print(i.quality)

4.0009750248505584
0.35580032087135716
3.377147216547643
3.0803743895907414
0.23860237809242102
9.133125542666312
4.228843962469185
3.5272633852617603
3.46033319800005
2.016305358621306
1.9211747111224717
3.156456110481663
1.1927105193704262
0.665917188012676
3.4976809163802725
2.0285982728598237
0.5013019315041888
5.050027581628739
3.6498157528709094
0.4929150642210218
3.0866208436809037
0.5645508163803665
0.49481551203183716
9.526970314322057
3.52392325736421
3.0547354830500963
19.036654638396087
3.2017270862186114
3.0419379665473976
3.1147766643923447
3.105871023785194
3.0747361002256794
4.455700693295404
2.327090878198261
1.0000000000000004
11.718758090987544
3.369540763534891
3.0715100502066908
16.596019368054883
4.527210701640863
3.3933873364694973
3.184863984387563
3.1352122987974296
3.1008895158253162
3.2710204666511578
2.735860521624798
2.443877908264569
3.0995664921909256
1.771397403387585
1.4873655563047572
5.4073134861864665
4.647331218335758
4.185333098008395
3.04527639407

In [469]:
k = len(kshapelets)
dataset = np.zeros((sample_nums, k))
for i, sp in enumerate(tqdm(kshapelets)):
    Ds = findDistances(sp, sp.length, a)
    dataset[:, i] = Ds
dataset = np.nan_to_num(dataset)
# print(dataset)
dataset_test = np.zeros((test_nums, k))
b = genDiffSeries(data_test)
for i, sp in enumerate(tqdm(kshapelets)):
    Ds = findDistances(sp, sp.length, b)
    dataset_test[:, i] = Ds
dataset_test = np.nan_to_num(dataset_test)
# from sklearn.linear_model import LogisticRegressionCV
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

sc = StandardScaler()
# scc = sc.fit_transform(dataset)
# np.set_printoptions(threshold=np.inf)
# print(dataset)
# print(scc)

# model = LogisticRegressionCV(max_iter=100000).fit(sc.fit_transform(dataset), target_train)
model = LinearSVC(max_iter=100000).fit(sc.fit_transform(dataset), target_train)
print(classification_report(target_train, model.predict(sc.transform(dataset))))
print(accuracy_score(target_train, model.predict(sc.transform(dataset))))
print(classification_report(target_test, model.predict(sc.transform(dataset_test))))
print(accuracy_score(target_test, model.predict(sc.transform(dataset_test))))

100%|██████████| 260/260 [00:10<00:00, 24.59it/s]

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         4
           1       1.00      1.00      1.00         4
           2       1.00      1.00      1.00         4

    accuracy                           1.00        12
   macro avg       1.00      1.00      1.00        12
weighted avg       1.00      1.00      1.00        12

1.0
              precision    recall  f1-score   support

           0       0.17      0.20      0.18         5
           1       0.40      0.40      0.40         5
           2       0.00      0.00      0.00         5

    accuracy                           0.20        15
   macro avg       0.19      0.20      0.19        15
weighted avg       0.19      0.20      0.19        15

0.2
